In [1]:
import os
from pathlib import Path

import pandas as pd

### Setup

In [2]:
os.chdir('..')
os.getcwd()

'c:\\Users\\delgr\\Projects\\advancedPython\\Day6a'

### Set Script Parameters

In [4]:
session_dir = './data/raw/20161214_cori_steinmetz'

In [5]:
session_path = Path(session_dir)
assert session_path.exists()

### Load Data

### Stimuli

In [9]:
stimuli = pd.read_csv(session_path/'stimuli.csv')
stimuli

,trial,contrast_left,contrast_right
0,1,100,0
1,2,0,50
2,3,100,50
3,4,0,0
4,5,50,100
...,...,...,...
359,360,50,25
360,361,50,25
361,362,0,50
362,363,25,0


#### Triggers


In [24]:
triggers = pd.read_csv(session_path/'triggers.dat', sep='\t', header=None, usecols=[0, 1], index_col=1, names=['label', 'time'])
triggers

,label
time,
0,TBEG
500,STIM
820,LICK
949,LICK
1027,GOCU
...,...
903000,STIM
905000,TBEG
905500,STIM


Signature: dict.setdefault(self, key, default=None, /)
Docstring:
Insert key with a value of default if key is not in the dictionary.

Return the value for key if key is in the dictionary, else default.
Type:      method_descriptor

In [77]:
trials = []
trial = None
stims = 0
for time, label in triggers['label'].to_dict().items():
    match label:
        case 'TBEG':
            ...  # Do Nothing
        case 'STIM': 
            if trial is not None:
                trials.append(trial)
            trial = {'stim_onset': time, 'licks': 0, 'response': None, 'feedback': None}
            stims += 1
        case 'LICK':
            trial['licks'] += 1            
        case 'GOCU': 
            trial['gocue'] = time
        case 'RESP': 
            trial['response_time'] = time
        case 'RSPL': 
            assert trial['response'] is None, trial['response']
            trial['response'] = 'left'
        case 'RSPR': 
            assert trial['response'] is None, trial['response']
            trial['response'] = 'right'
        case 'RSPN': 
            trial['response'] = 'neutral'
        case 'FPOS': 
            assert trial['feedback'] is None, trial['feedback']
            trial['feedback'] = 'pos'
        case 'FNEG':
            assert trial['feedback'] is None, trial['feedback']
            trial['feedback'] = 'neg'
        case _: raise ValueError(label)
trials.append(trial)

stims, pd.DataFrame(trials)

(364,
      stim_onset  licks response feedback    gocue  response_time
 0           500     11    right      pos   1027.0         1610.0
 1          3000     12     left      pos   3374.0         3070.0
 2          5500     13    right      pos   5825.0         6170.0
 3          8000      0  neutral      pos   8261.0        10130.0
 4         10500      7    right      neg  10662.0            NaN
 ..          ...    ...      ...      ...      ...            ...
 359      898000      0     None     None      NaN            NaN
 360      900500      0     None     None      NaN            NaN
 361      903000      0     None     None      NaN            NaN
 362      905500      0     None     None      NaN            NaN
 363      908000      0     None     None      NaN            NaN
 
 [364 rows x 6 columns])

### Combine stimuli and trials

In [102]:
import numpy as np


assert len(stimuli) == len(trials)
trials_df = pd.DataFrame(trials)
trials_all = pd.merge(stimuli, trials_df, left_index=True, right_index=True)
trials_all = trials_all.astype({
    'trial': np.uint16, 
    'contrast_left': np.uint8, 
    'contrast_right': np.uint8, 
    'licks': np.uint8,
    'response': 'S',
    'feedback': 'S',
})
trials_all


,trial,contrast_left,contrast_right,stim_onset,licks,response,feedback,gocue,response_time
0,1,100,0,500,11,b'right',b'pos',1027.0,1610.0
1,2,0,50,3000,12,b'left',b'pos',3374.0,3070.0
2,3,100,50,5500,13,b'right',b'pos',5825.0,6170.0
3,4,0,0,8000,0,b'neutral',b'pos',8261.0,10130.0
4,5,50,100,10500,7,b'right',b'neg',10662.0,NaN
...,...,...,...,...,...,...,...,...,...
359,360,50,25,898000,0,b'None',b'None',NaN,NaN
360,361,50,25,900500,0,b'None',b'None',NaN,NaN
361,362,0,50,903000,0,b'None',b'None',NaN,NaN
362,363,25,0,905500,0,b'None',b'None',NaN,NaN


In [103]:
trials_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   trial           364 non-null    uint16 
 1   contrast_left   364 non-null    uint8  
 2   contrast_right  364 non-null    uint8  
 3   stim_onset      364 non-null    int64  
 4   licks           364 non-null    uint8  
 5   response        364 non-null    |S7    
 6   feedback        364 non-null    |S4    
 7   gocue           211 non-null    float64
 8   response_time   176 non-null    float64
dtypes: bytes32(1), bytes56(1), float64(2), int64(1), uint16(1), uint8(3)
memory usage: 14.3 KB


In [104]:
trials_all.to_records()

rec.array([(  0,   1, 100,   0,    500, 11, b'right', b'pos',   1027.,   1610.),
           (  1,   2,   0,  50,   3000, 12, b'left', b'pos',   3374.,   3070.),
           (  2,   3, 100,  50,   5500, 13, b'right', b'pos',   5825.,   6170.),
           (  3,   4,   0,   0,   8000,  0, b'neutral', b'pos',   8261.,  10130.),
           (  4,   5,  50, 100,  10500,  7, b'right', b'neg',  10662.,     nan),
           (  5,   6,   0,   0,  13000,  3, b'right', b'neg',  13686.,  14280.),
           (  6,   7,   0,   0,  15500,  0, b'left', b'neg',  15588.,  17870.),
           (  7,   8,   0,   0,  18000,  0, b'left', b'neg',  18106.,     nan),
           (  8,   9,   0,   0,  20500,  0, b'neutral', b'pos',  20591.,  21220.),
           (  9,  10, 100,  50,  23000, 14, b'right', b'pos',  23571.,  23180.),
           ( 10,  11,  50,   0,  25500, 14, b'right', b'pos',  25646.,  25710.),
           ( 11,  12,   0,   0,  28000,  0, b'neutral', b'pos',  28681.,  28180.),
           ( 12,  13,  50

In [106]:
extract_path = Path(f'data/extracted/{session_path.name}/trials')
extract_path.mkdir(exist_ok=True, parents=True)
np.save(extract_path/'trials.npy', trials_all.to_records(index=False), allow_pickle=False)